In [ ]:
import numpy as np
from gtsam import symbol
from gtsam import Pose2
from gtsam import Values
from gtsam import Marginals
from gtsam import noiseModel
from gtsam import PriorFactorPose2
from gtsam import BetweenFactorPose2
from gtsam import NonlinearFactorGraph
from gtsam import GaussNewtonParams
from gtsam import GaussNewtonOptimizer

In [ ]:
# create an empty nonlinear factor graph
graph = NonlinearFactorGraph()

# add a prior on the first pose, setting it to the origin
# the prior is needed to fix/align the whole trajectory at world frame
# a prior factor consists of a mean value and a noise model (covariance matrix)
prior_noise = noiseModel.Diagonal.Sigmas(np.array([1.0, 1.0, 0.1])) #prior loss function
prior_mean = Pose2(0.0, 0.0 ,0.0)
graph.add(PriorFactorPose2(symbol('x', 1), prior_mean, prior_noise))


# odometry measurement noise model (covariance matrix)
odom_model = noiseModel.Diagonal.Sigmas(np.array([0.5, 0.5, 0.1]))

# add odometry factors
# create odometry (Between) factors between consecutive poses
# robot makes 90 deg right turns at x3 - x5
graph.add(BetweenFactorPose2(symbol('x', 1), symbol('x', 2), Pose2(5, 0, 0), odom_model))
graph.add(BetweenFactorPose2(symbol('x', 2), symbol('x', 3), Pose2(5, 0, -np.pi/2), odom_model))
graph.add(BetweenFactorPose2(symbol('x', 3), symbol('x', 4), Pose2(5, 0, -np.pi/2), odom_model))
graph.add(BetweenFactorPose2(symbol('x', 4), symbol('x', 5), Pose2(5, 0, -np.pi/2), odom_model))

# loop closure measurement noise model
loop_model = noiseModel.Diagonal.Sigmas(np.array([0.5, 0.5, 0.1]))


# add the loop closure constraint
graph.add(BetweenFactorPose2(symbol('x', 5), symbol('x', 2), \
                             Pose2(5, 0, -np.pi/2), loop_model))

print("\nGraph:\n", graph)

initials = Values()
initials.insert(symbol('x', 1), Pose2(0.2, -0.3, 0.2))
initials.insert(symbol('x', 2), Pose2(5.1, 0.3, -0.1))
initials.insert(symbol('x', 3), Pose2(9.9, -0.1, -np.pi/2 - 0.2))
initials.insert(symbol('x', 4), Pose2(10.2, -5.0, -np.pi + 0.1))
initials.insert(symbol('x', 5), Pose2(5.1, -5.1, np.pi/2 - 0.1))

print("\nInitial Values:\n", initials)

# create Gauss-Newton optimizer
parameters = GaussNewtonParams()
parameters.setVerbosity("ERROR")
optimizer = GaussNewtonOptimizer(graph, initials, parameters)

# optimize the factor graph
results = Values()
results = optimizer.optimize()

print("\nFinal Result:\n", results)


marginals = Marginals(graph, results)
print("x1 covariance:\n", marginals.marginalCovariance(symbol('x', 1)))
print("x2 covariance:\n", marginals.marginalCovariance(symbol('x', 2)))
print("x3 covariance:\n", marginals.marginalCovariance(symbol('x', 3)))
print("x4 covariance:\n", marginals.marginalCovariance(symbol('x', 4)))
print("x5 covariance:\n", marginals.marginalCovariance(symbol('x', 5)))
